In [4]:
import warnings
warnings.filterwarnings('ignore')
%run Functions.py

In [5]:
train = pd.read_csv('data/train_c.csv')
test = pd.read_csv('data/test_c.csv')

In [6]:
train,test = get_additional_features(train,test,magic=False)

## Model 1: GBDT

In [8]:
from sklearn.ensemble import GradientBoostingRegressor
col = test.columns
gb1 = GradientBoostingRegressor(n_estimators=1000,max_features=0.95,learning_rate=0.005,max_depth=4)
gb1_train,gb1_test = get_sklearn_stack_data(gb1,train,col,train['y'],test)

Training 1 Fold
R2 Scored of Fold 1 is -2.51468699269
RMSE of Fold 1 is 23.1664457022
Training 2 Fold
R2 Scored of Fold 2 is 0.42550157235
RMSE of Fold 2 is 10.8269723747
Training 3 Fold
R2 Scored of Fold 3 is 0.571767204667
RMSE of Fold 3 is 8.07903738315
Training 4 Fold
R2 Scored of Fold 4 is 0.529137429404
RMSE of Fold 4 is 8.64125312768
Training 5 Fold
R2 Scored of Fold 5 is 0.249526882522
RMSE of Fold 5 is 9.95192409346
Start Training
Calculating In-Bag R2 Score
0.677339583666
Calculating Out-Bag R2 Score
-0.147750780749
Calculating In-Bag RMSE
7.20143938119
Calculating Out-Bag RMSE
12.1331265362


## Model 2: lgb

In [10]:
params = {
            'objective': 'regression',
            'metric': 'rmse',
            'boosting': 'gbdt',
            'learning_rate': 0.0045 , #small learn rate, large number of iterations
            'verbose': 0,
            'num_iterations': 500,
            'bagging_fraction': 0.95,
            'bagging_freq': 1,
            'bagging_seed': 42,
            'feature_fraction': 0.95,
            'feature_fraction_seed': 42,
            'max_bin': 100,
            'max_depth': 3,
            'num_rounds': 1000
        }
lgb_train, lgb_test = get_lgb_stack_data(params,1000,train,col,train['y'],test)

Training 1 Fold
R2 Scored of Fold 1 is 0.392935226846
RMSE of Fold 1 is 9.62794490979
Training 2 Fold
R2 Scored of Fold 2 is 0.445326565495
RMSE of Fold 2 is 10.6385218432
Training 3 Fold
R2 Scored of Fold 3 is 0.595935251881
RMSE of Fold 3 is 7.84774959608
Training 4 Fold
R2 Scored of Fold 4 is 0.537189216766
RMSE of Fold 4 is 8.56705148892
Training 5 Fold
R2 Scored of Fold 5 is 0.610236463069
RMSE of Fold 5 is 7.17199682513
Start Training
Calculating In-Bag R2 Score
0.589304339798
Calculating Out-Bag R2 Score
0.516324544811
Calculating In-Bag RMSE
8.12468479788
Calculating Out-Bag RMSE
8.77065293262


## Model3: xgb

In [14]:
y_mean = np.mean(train.y)
xgb_params = {
        'n_trees': 520, 
        'eta': 0.0045,
        'max_depth': 4,
        'subsample': 0.93,
        'objective': 'reg:linear',
        'eval_metric': 'rmse',
        'base_score': y_mean, # base prediction = mean(target)
        'silent': True,
        'seed': 42,
    }
xgb_train, xgb_test = get_xgb_stack_data(xgb_params,1250,train,col,train['y'],test)

Training 1 Fold
R2 Scored of Fold 1 is -3.28989812295
RMSE of Fold 1 is 25.5940827511
Training 2 Fold
R2 Scored of Fold 2 is 0.433195031888
RMSE of Fold 2 is 10.7542327267
Training 3 Fold
R2 Scored of Fold 3 is 0.586389170167
RMSE of Fold 3 is 7.93991049281
Training 4 Fold
R2 Scored of Fold 4 is 0.524419669288
RMSE of Fold 4 is 8.68443531519
Training 5 Fold
R2 Scored of Fold 5 is 0.375149269854
RMSE of Fold 5 is 9.08087349568
Start Training
Calculating In-Bag R2 Score
0.635691675237
Calculating Out-Bag R2 Score
-0.274148996351
Calculating In-Bag RMSE
7.65210646008
Calculating Out-Bag RMSE
12.4107069563


## Model 4: Lasso with Magic

In [18]:
train = pd.read_csv('data/train_c.csv')
test = pd.read_csv('data/test_c.csv')
train,test = get_additional_features(train,test,magic=True)
col = test.columns

In [19]:
from sklearn.linear_model import Lasso

In [20]:
las1 = Lasso(alpha=5,random_state=42)
las1_train,las1_test = get_sklearn_stack_data(las1,train,col,train['y'],test)

Training 1 Fold
R2 Scored of Fold 1 is 0.626692158964
RMSE of Fold 1 is 7.55004667508
Training 2 Fold
R2 Scored of Fold 2 is 0.451603223839
RMSE of Fold 2 is 10.578158084
Training 3 Fold
R2 Scored of Fold 3 is 0.604652087853
RMSE of Fold 3 is 7.76263883468
Training 4 Fold
R2 Scored of Fold 4 is 0.544009741963
RMSE of Fold 4 is 8.50369009623
Training 5 Fold
R2 Scored of Fold 5 is 0.641373063165
RMSE of Fold 5 is 6.87956436386
Start Training
Calculating In-Bag R2 Score
0.575435112803
Calculating Out-Bag R2 Score
0.573666055157
Calculating In-Bag RMSE
8.26073140263
Calculating Out-Bag RMSE
8.25481961077


## Try: Stacking four models with xgb

In [21]:
stack_train = gb1_train[['label','predicted']]
stack_train.columns = ['y','gbdt']
stack_train['las'] = las1_train['predicted']
stack_train['lgb'] = lgb_train['predicted']
stack_train['xgb'] = xgb_train['predicted']

In [22]:
stack_test = pd.DataFrame({'gbdt':gb1_test['y'],'xgb':xgb_test['y'],'las':las1_test['y'],'lgb':lgb_test['y']})

In [23]:
col = stack_test.columns

In [24]:
params = {
    'eta': 0.005,
    'max_depth': 2,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'base_score': y_mean, # base prediction = mean(target)
    'silent': 1
}

In [25]:
dtrain = xgb.DMatrix(stack_train[col],stack_train.y)
dtest = xgb.DMatrix(stack_test)
xgb_cvalid = xgb.cv(params, dtrain, num_boost_round=2000, early_stopping_rounds=20,
    verbose_eval=50, show_stdv=True,seed=42)
print('Performance does not improve from '+str(len(xgb_cvalid))+' rounds')

[0]	train-rmse:12.641+0.179047	test-rmse:12.6376+0.352979
[50]	train-rmse:11.154+0.221898	test-rmse:11.1584+0.416284
[100]	train-rmse:10.1442+0.257133	test-rmse:10.1553+0.472986
[150]	train-rmse:9.47778+0.283096	test-rmse:9.49482+0.520715
[200]	train-rmse:9.04766+0.302162	test-rmse:9.07161+0.556564
[250]	train-rmse:8.77171+0.314994	test-rmse:8.80443+0.581305
[300]	train-rmse:8.59297+0.322364	test-rmse:8.63845+0.596148
[350]	train-rmse:8.47518+0.325443	test-rmse:8.5362+0.608529
[400]	train-rmse:8.39667+0.326192	test-rmse:8.47117+0.619434
[450]	train-rmse:8.34257+0.326162	test-rmse:8.42987+0.627002
[500]	train-rmse:8.30412+0.325737	test-rmse:8.40322+0.632197
[550]	train-rmse:8.27598+0.325296	test-rmse:8.3858+0.636377
[600]	train-rmse:8.25015+0.324509	test-rmse:8.37363+0.639779
[650]	train-rmse:8.22786+0.322959	test-rmse:8.36508+0.641881
[700]	train-rmse:8.1971+0.316296	test-rmse:8.3583+0.644187
[750]	train-rmse:8.17349+0.314434	test-rmse:8.35344+0.646074
[800]	train-rmse:8.16144+0.317999

In [27]:
model = xgb.train(params,dtrain,num_boost_round=1031)

In [28]:
pred = model.predict(dtest)

In [29]:
sub= pd.DataFrame({'ID':test['ID'],'y':pred})
sub.to_csv('submission/stacking_four_models_with_xgb_together.csv',index=False)

## Stack First Three with xgb then average with independent xgb

In [30]:
del stack_train['xgb']
del stack_test['xgb']

In [31]:
col = stack_test.columns

In [32]:
dtrain = xgb.DMatrix(stack_train[col],stack_train.y)
dtest = xgb.DMatrix(stack_test)
xgb_cvalid = xgb.cv(params, dtrain, num_boost_round=2000, early_stopping_rounds=20,
    verbose_eval=50, show_stdv=True,seed=42)
print('Performance does not improve from '+str(len(xgb_cvalid))+' rounds')

[0]	train-rmse:12.641+0.179047	test-rmse:12.6376+0.352979
[50]	train-rmse:11.154+0.221898	test-rmse:11.1584+0.416284
[100]	train-rmse:10.1442+0.257133	test-rmse:10.1553+0.472986
[150]	train-rmse:9.47778+0.283096	test-rmse:9.49482+0.520715
[200]	train-rmse:9.04766+0.302162	test-rmse:9.07161+0.556564
[250]	train-rmse:8.77171+0.314994	test-rmse:8.80443+0.581305
[300]	train-rmse:8.59298+0.322354	test-rmse:8.63837+0.596031
[350]	train-rmse:8.47524+0.32536	test-rmse:8.53575+0.607896
[400]	train-rmse:8.39687+0.325916	test-rmse:8.47004+0.617829
[450]	train-rmse:8.34308+0.325365	test-rmse:8.42818+0.624992
[500]	train-rmse:8.30503+0.324314	test-rmse:8.4005+0.630758
[550]	train-rmse:8.27748+0.323413	test-rmse:8.38275+0.635503
[600]	train-rmse:8.256+0.323437	test-rmse:8.37114+0.639964
[650]	train-rmse:8.2349+0.322743	test-rmse:8.36397+0.642319
[700]	train-rmse:8.2128+0.321939	test-rmse:8.35807+0.644216
[750]	train-rmse:8.19621+0.323724	test-rmse:8.35435+0.64558
[800]	train-rmse:8.18461+0.328669	te

In [35]:
model = xgb.train(params,dtrain,num_boost_round=1012)
pred = model.predict(dtest)

In [37]:
Average = xgb_test.y*0.73 + pred*0.27
sub = pd.DataFrame({'ID':test['ID'],'y':Average})

In [38]:
sub.to_csv('submission/Stack_then_Average_sklearn_xgb_Tree_no_magic.csv',index=False)

## Add a little tensorflow submission?

In [43]:
y1 = sub.y
y2 = pd.read_csv('submission/tensorflow_results.csv').y

In [44]:
Average = 0.85*y1 + 0.15*y2

In [45]:
sub = pd.DataFrame({'ID':test['ID'],'y':Average})
sub.to_csv('submission/Stack_sklearn_average_xgb_average_tf.csv',index=False)